In [224]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances, paired_distances
from sklearn.metrics import pairwise_distances
import numpy as np
from scipy import spatial

In [225]:
mutational_matrix = pd.read_csv(r'./resources/mut_mat.csv')
reconstructed_matrix = pd.read_csv(r'./resources/reconstructed.csv')
cosmic_catalog = pd.read_csv(r'./resources/cosmic_mutations.csv', delimiter='\t')
contribution_matrix = pd.read_csv(r'./resources/contribution.csv')

In [226]:
mutational_matrix = mutational_matrix.sort_values(by='Somatic Mutation Type', axis=0)
reconstructed_matrix = reconstructed_matrix.sort_values(by='Somatic Mutation Type', axis=0)
cosmic_catalog = cosmic_catalog.sort_values(by='Somatic Mutation Type', axis=0)

In [227]:
cosmic_catalog = cosmic_catalog.loc[:, ~cosmic_catalog.columns.str.contains('^Unnamed')]

In [309]:
all = ['colon1','colon2','colon3','intestine1','intestine2','intestine3','liver1','liver2','liver3']
selected = ['liver3']

In [310]:
my_input = mutational_matrix[all].values
my_reconstructed = reconstructed_matrix[all].values
my_contribution = contribution_matrix[all].values

In [311]:
my_catalog = cosmic_catalog.loc[:, cosmic_catalog.columns.str.contains('^Signature')].values

In [312]:
new_matrix = np.dot(my_catalog, my_contribution)

In [313]:
similarity = paired_distances(new_matrix.reshape(-1,1), my_input.reshape(-1,1), metric='manhattan')

In [314]:
np.average(similarity)

1.5375217392027791

In [315]:
np.sum(similarity)

1328.4187826712011

In [11]:
signeR_matrix = pd.read_csv(r'./resources/21_breast_cancers.mutations.txt', delimiter='\t')
signeR_contribution = pd.read_csv(r'./resources/exposures_signeR.csv')
signeR_signatures = pd.read_csv(r'./resources/signatures_signeR.csv')

In [12]:
signeR_matrix = signeR_matrix.loc[:, ~signeR_matrix.columns.str.contains('^Unnamed')].values
signeR_contribution = signeR_contribution.loc[:, ~signeR_contribution.columns.str.contains('^Unnamed')].values
signeR_signatures = signeR_signatures.loc[:, ~signeR_signatures.columns.str.contains('^Unnamed')].values

In [101]:
signeR_contribution.shape

(5, 21)

In [13]:
signeR_reconstructed = np.dot(signeR_signatures, signeR_contribution)

In [14]:
signeR_reconstructed = signeR_reconstructed.transpose()

In [15]:
signeR_reconstructed.shape

(21, 96)

In [16]:
signeR_matrix.shape

(21, 96)

In [316]:
signeR_similarity = paired_distances(signeR_reconstructed.reshape(-1,1), signeR_matrix.reshape(-1,1), metric='manhattan')

In [317]:
np.average(signeR_similarity)

6.383718145214449

In [318]:
np.sum(signeR_similarity)

12869.57578075233

In [195]:
matrix = signeR_matrix.transpose()

In [196]:
from sklearn.decomposition import NMF

In [214]:
model = NMF(n_components=30, solver='cd', beta_loss='frobenius', random_state=0, init='random')
W = model.fit_transform(matrix)
H = model.components_

In [215]:
diviser = np.max(W)
W = W * (1 / diviser)
H = H * diviser

In [216]:
reconstructed_NMF = np.dot(W, H)

In [217]:
NMF_similarity = paired_distances(reconstructed_NMF.reshape(-1,1), matrix.reshape(-1,1), metric='manhattan')

In [222]:
np.average(NMF_similarity)

0.37675544084965124

In [223]:
np.sum(NMF_similarity)

759.5389687528968